# Clustering of the neighborhoods in Toronto

### 1. Gathering Data

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
import requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  --> 2019.9.11-py36_0  conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.0.2t-h7b6447c_1 --> 1.0.2r-h14c3975_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of cond

In [12]:
# get data from wikipedia and create a soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'html.parser')

### 2. Creating a Dataframe and Cleaning the Data

In [13]:
#use our soup object to extract wikitable data from html page and store it into a list.
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #set first row of the list to be its header.
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into pd DataFrame.
ca_df = pd.DataFrame(data = data,columns = columns)
ca_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
#create a new dataframe which ignores Neighbourhoods that are labeled as Not assigned.
df1=ca_df
ca_df1 = df1.loc[df1['Neighbourhood']== 'Not assigned']

#replace Not assigned Neighbourhoods column values with values from Borough.
for i in ca_df1.index:
    ca_df1.at[i, 'Neighbourhood'] = ca_df1.at[i, 'Borough']

#replace rows in original DataFrame.
df1.loc[ca_df1.index] = ca_df1

#delete rows with Borough that are Not assigned.
ca_df = df1[df1.Borough != 'Not assigned']

#group by postcode and Borough, keeping all the Neighbourhoods.
ca_df=ca_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()

#remove duplicates.
ca_df = ca_df.drop_duplicates()

#print our DataFrame.
ca_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### 3. Print the number of rows of the Dataframe

In [15]:
ca_df.shape

(103, 3)

### 4. Adding Geospatical Data

In [16]:
!conda install geopandas --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 
    openssl: 1.0.2r-h14c3975_0 conda-forge --> 1.0.2t-h7b6447c_1


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [17]:
#read and load geographical locations of Toronto postcodes.
pc_geo = pd.read_csv('https://cocl.us/Geospatial_data', header=0)
pc_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)

#merge postcodes list within Borough & Neigbourhood DataFrame.
pc_m_geo=pd.merge(ca_df,pc_geo, on='Postcode')

#print our updated DataFrame.
pc_m_geo

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### 5. Exploration of Toronto neighborhoods

In [ ]:
# get Toronto City's coordinates
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

In [20]:
# verifying Toronto's coordinates from google search
latitude = 43.6532
longitude = -79.3832

# create a map of Toronto neighborhoods using their latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(pc_m_geo['Latitude'], pc_m_geo['Longitude'], pc_m_geo['Borough'], pc_m_geo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [42]:
# Define Foursquare Credentials
CLIENT_ID = '3MBRT1YWPA41ODSPKR4KMREK0R0FTH2AMJ5X15MVMCFDIGMV' # your Foursquare ID
CLIENT_SECRET = 'AYUT45FXBXZD4BAMYNRZ2QHNPPZ4XWW0MGXRCNZQ13TSPIC5' # your Foursquare Secret
VERSION = '20191028' # Foursquare API version

In [43]:
# Get the name of the category
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# taking foursquare information for all neighborhoods
explore_df_list = []

for i, nbd_name in enumerate(pc_m_geo['Neighbourhood']):  
    
    try :
        ### Getting the data of neighbourhood
        nbd_name = pc_m_geo.loc[i, 'Neighbourhood']
        nbd_lat = pc_m_geo.loc[i, 'Latitude']
        nbd_lng = pc_m_geo.loc[i, 'Longitude']

        radius = 2000 # Setting the radius as 2km
        LIMIT = 20 # Getting the top 20 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

In [44]:
# create a dataframe for clustering
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,"Rouge,Malvern",43.806686,-79.194353,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
1,"Rouge,Malvern",43.806686,-79.194353,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
2,"Rouge,Malvern",43.806686,-79.194353,Images Salon & Spa,Spa,43.802283,-79.198565
3,"Rouge,Malvern",43.806686,-79.194353,Toronto Zoo,Zoo,43.820582,-79.181551
4,"Rouge,Malvern",43.806686,-79.194353,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374


In [45]:
# some pre-processing of the above Dataframe

# One hot encoding
toronto_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = explore_df['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + toronto_onehot.columns[:-1].values.tolist()
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool Hall,Portuguese Restaurant,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [46]:
# grouping venues by neighbourhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool Hall,Portuguese Restaurant,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.05,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.

In [49]:
# function to return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# getting the 10 most commmon venues for each neighbourhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",American Restaurant,Gym / Fitness Center,Thai Restaurant,Pizza Place,Plaza,Neighborhood,Restaurant,Mediterranean Restaurant,Seafood Restaurant,Smoke Shop
1,Agincourt,Chinese Restaurant,Gym / Fitness Center,Breakfast Spot,Caribbean Restaurant,Bakery,Korean Restaurant,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Noodle House,Bubble Tea Shop,Indian Restaurant,Hong Kong Restaurant,Tattoo Parlor,Ice Cream Shop,Bakery,Caribbean Restaurant,Japanese Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Coffee Shop,Pizza Place,Grocery Store,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Skating Rink,Steakhouse,Beer Store,Fried Chicken Joint
4,"Alderwood,Long Branch",Coffee Shop,Burger Joint,Restaurant,Café,Bookstore,Seafood Restaurant,Shopping Mall,Toy / Game Store,Pub,Electronics Store


In [76]:
# clustering of neighbourhoods, by using K-Means Algorithm

#setting the dataframe for K-Means clustering
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# we use K=4 clusters
kclusters = 4

# Run k-means clustering
kgc = toronto_grouped_clustering
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', random_state = 0).fit(kgc)

In [77]:
# Add clustering labels
#neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# final dataframe, containing all the appropriate data
toronto_merged = pc_m_geo
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3,Zoo Exhibit,Zoo,Athletics & Sports,Spa,Fried Chicken Joint,Paper / Office Supplies Store,Caribbean Restaurant,Liquor Store,Fast Food Restaurant,Falafel Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4,Pharmacy,Breakfast Spot,Pizza Place,Neighborhood,Supermarket,Beer Store,Italian Restaurant,Fried Chicken Joint,Mexican Restaurant,Ice Cream Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1,Pizza Place,Fast Food Restaurant,Bank,Park,Coffee Shop,Discount Store,Sandwich Place,Smoothie Shop,Liquor Store,Pharmacy
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Indian Restaurant,Coffee Shop,Pub,Fast Food Restaurant,Juice Bar,Steakhouse,Music Store,Chinese Restaurant,Clothing Store,Bank
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Indian Restaurant,Steakhouse,Gym / Fitness Center,Park,Hakka Restaurant,Coffee Shop,Clothing Store,Chinese Restaurant,Music Store,Caribbean Restaurant


In [78]:
# visualizing the K clusters

# Create map of Toronto city, using its coordinates
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster + 1) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

In [79]:
# examining cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + np.arange(4, toronto_merged.shape[1]).tolist()]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,-79.188711,1,Pizza Place,Fast Food Restaurant,Bank,Park,Coffee Shop,Discount Store,Sandwich Place,Smoothie Shop,Liquor Store,Pharmacy
3,M1G,-79.216917,1,Indian Restaurant,Coffee Shop,Pub,Fast Food Restaurant,Juice Bar,Steakhouse,Music Store,Chinese Restaurant,Clothing Store,Bank
5,M1J,-79.239476,1,Coffee Shop,Sandwich Place,Gym,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Butcher,Liquor Store,Bookstore,Dog Run
6,M1K,-79.262029,1,Coffee Shop,Chinese Restaurant,Gym / Fitness Center,Bank,Pet Store,Deli / Bodega,Department Store,Sandwich Place,Discount Store,Liquor Store
7,M1L,-79.284577,1,Coffee Shop,Discount Store,Chinese Restaurant,Sandwich Place,Movie Theater,Burger Joint,Indian Restaurant,Restaurant,Bakery,Gas Station
8,M1M,-79.239476,1,Park,Sandwich Place,Coffee Shop,Beach,Bank,Fish & Chips Shop,Breakfast Spot,Pharmacy,Sporting Goods Shop,Sports Bar
9,M1N,-79.264848,1,Coffee Shop,Fish & Chips Shop,General Entertainment,Filipino Restaurant,Breakfast Spot,Café,Sushi Restaurant,Burger Joint,Thai Restaurant,Restaurant
10,M1P,-79.273304,1,Coffee Shop,Fish Market,Indian Restaurant,Park,Sandwich Place,Chinese Restaurant,Electronics Store,Burrito Place,Burger Joint,Pharmacy
24,M2R,-79.442259,1,Middle Eastern Restaurant,Coffee Shop,Pizza Place,Korean Restaurant,Pharmacy,Auto Dealership,Doner Restaurant,Bank,Bagel Shop,Chinese Restaurant
28,M3H,-79.442259,1,Coffee Shop,Sandwich Place,Deli / Bodega,Fast Food Restaurant,Sushi Restaurant,Bank,Restaurant,Bagel Shop,Middle Eastern Restaurant,Diner


In [80]:
# examining cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + np.arange(4, toronto_merged.shape[1]).tolist()]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,M3B,-79.352188,2,Café,Supermarket,Mexican Restaurant,Coffee Shop,Movie Theater,Mediterranean Restaurant,Caribbean Restaurant,Cantonese Restaurant,Seafood Restaurant,Bagel Shop
27,M3C,-79.340923,2,Restaurant,Ice Cream Shop,Bakery,Café,Middle Eastern Restaurant,Shopping Mall,Movie Theater,Science Museum,Japanese Restaurant,Italian Restaurant
41,M4K,-79.352188,2,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Concert Hall,Bakery,Brewery,Spa,Pizza Place,Bookstore,Tapas Restaurant
42,M4L,-79.315572,2,Brewery,Café,Beach,Ice Cream Shop,Middle Eastern Restaurant,Farmers Market,Egyptian Restaurant,Italian Restaurant,Burger Joint,Coffee Shop
45,M4P,-79.390197,2,Italian Restaurant,Café,Coffee Shop,Bookstore,Bakery,Yoga Studio,Tea Room,Supermarket,Park,Vegetarian / Vegan Restaurant
46,M4R,-79.405678,2,Italian Restaurant,Yoga Studio,Café,Coffee Shop,Tea Room,Flower Shop,Food & Drink Shop,Supermarket,Garden,Bakery
47,M4S,-79.388790,2,Italian Restaurant,Café,Gym,Ramen Restaurant,Bookstore,Steakhouse,Supermarket,Indonesian Restaurant,Indian Restaurant,Dessert Shop
48,M4T,-79.383160,2,Park,Spa,Italian Restaurant,Coffee Shop,Café,Juice Bar,Sushi Restaurant,Tapas Restaurant,Liquor Store,Bagel Shop
49,M4V,-79.400049,2,Spa,Italian Restaurant,Liquor Store,Bagel Shop,Juice Bar,Café,Supermarket,Sushi Restaurant,French Restaurant,Castle
50,M4W,-79.377529,2,Park,Coffee Shop,Spa,Historic Site,Liquor Store,Juice Bar,Italian Restaurant,Salon / Barbershop,Thai Restaurant,Bar


In [81]:
# examining cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + np.arange(4, toronto_merged.shape[1]).tolist()]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,3,Zoo Exhibit,Zoo,Athletics & Sports,Spa,Fried Chicken Joint,Paper / Office Supplies Store,Caribbean Restaurant,Liquor Store,Fast Food Restaurant,Falafel Restaurant


In [82]:
# examining cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + np.arange(4, toronto_merged.shape[1]).tolist()]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,-79.160497,4,Pharmacy,Breakfast Spot,Pizza Place,Neighborhood,Supermarket,Beer Store,Italian Restaurant,Fried Chicken Joint,Mexican Restaurant,Ice Cream Shop
4,M1H,-79.239476,4,Indian Restaurant,Steakhouse,Gym / Fitness Center,Park,Hakka Restaurant,Coffee Shop,Clothing Store,Chinese Restaurant,Music Store,Caribbean Restaurant
11,M1R,-79.295849,4,Middle Eastern Restaurant,Burger Joint,Coffee Shop,Mediterranean Restaurant,Liquor Store,Pizza Place,Korean Restaurant,Seafood Restaurant,Fish Market,Supermarket
12,M1S,-79.262029,4,Chinese Restaurant,Gym / Fitness Center,Breakfast Spot,Caribbean Restaurant,Bakery,Korean Restaurant,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket
13,M1T,-79.304302,4,Falafel Restaurant,Chinese Restaurant,Fast Food Restaurant,Noodle House,Burrito Place,Pharmacy,Seafood Restaurant,Cantonese Restaurant,Sushi Restaurant,Caribbean Restaurant
14,M1V,-79.284577,4,Chinese Restaurant,Noodle House,Bubble Tea Shop,Indian Restaurant,Hong Kong Restaurant,Tattoo Parlor,Ice Cream Shop,Bakery,Caribbean Restaurant,Japanese Restaurant
15,M1W,-79.318389,4,Chinese Restaurant,Caribbean Restaurant,Korean Restaurant,Japanese Restaurant,Tattoo Parlor,Mexican Restaurant,Hotpot Restaurant,Bakery,BBQ Joint,Beer Store
16,M1X,-79.205636,4,Sculpture Garden,Playground,Farm,Grocery Store,Golf Course,Trail,Electronics Store,Dog Run,Doner Restaurant,Eastern European Restaurant
17,M2H,-79.363452,4,Chinese Restaurant,Bakery,Bubble Tea Shop,Cantonese Restaurant,Japanese Restaurant,Taiwanese Restaurant,Coffee Shop,Korean Restaurant,Sushi Restaurant,BBQ Joint
18,M2J,-79.346556,4,Middle Eastern Restaurant,Breakfast Spot,Tea Room,Department Store,Chinese Restaurant,Salon / Barbershop,Electronics Store,Movie Theater,Caribbean Restaurant,Candy Store
